## Deriving the Covariance relationships of Depolarizing Channel

Given the Hamiltonain only depends on $R_xy$
Where:
$$R_xy = cos(\phi) X + sin(\phi) Y$$

and 
$$ U = cos(\theta s /2) I - i sin(\theta s /2) R $$

In [1]:
import sympy as sp
import numpy as np
from numpy import exp, sqrt, array
from numpy.random import multivariate_normal

In [2]:
def parameters(pp, pT1, pT2, ptg):
    p = pp                  # depolarizing probability
    T1 = pT1                 # relaxation time (seconds)
    T2 = pT2                 # dephasing time (seconds)
    tg = ptg                 # gate time (seconds)

    # Derived noise amplitudes
    ed = sqrt(p / 4)
    e1 = sqrt(tg / T1) if T1 else 0
    e2 = sqrt(tg / T2) if T2 else 0
    ep = sqrt(0.5 * (e2**2 - e1**2 / 2)) if T2 else 0
    return ed, e1, e2, ep

In [3]:

# === Symbols ===
theta, s, phi = sp.symbols('theta s phi', real=True)
alpha = s * theta / 2

i = sp.I

# === Pauli Matrices ===
I2 = sp.eye(2)
X = sp.Matrix([[0, 1], [1, 0]])
Y = sp.Matrix([[0, -i], [i, 0]])
Z = sp.Matrix([[1, 0], [0, -1]])
pauli_labels = {'I': I2, 'X': X, 'Y': Y, 'Z': Z}

# === Rotation Axes ===
R_xy = sp.cos(phi) * X + sp.sin(phi) * Y
R = R_xy

# === IBM-style unitary ===
U = sp.cos(alpha) * I2 - sp.I * sp.sin(alpha) * R
U_dag = U.H

# === Core function: compute f_k^σ(s) components ===
def compute_pauli_components(sigma_k):
    Lk = sp.simplify(U_dag * sigma_k * U)
    components = {}
    for label, P in pauli_labels.items():
        f_σ = sp.simplify(sp.trigsimp((Lk * P).trace() / 2))
        components[label] = f_σ
    return Lk, components

# === Integration helper for variances and covariances ===
def compute_var_and_cov(f_dict):
    keys = list(f_dict.keys())
    results = {}
    for k in keys:
        results[f'Var_{k}'] = sp.simplify(sp.integrate(f_dict[k]**2, (s, 0, 1)))
    for i in range(len(keys)):
        for j in range(i+1, len(keys)):
            k1, k2 = keys[i], keys[j]
            cov = sp.simplify(sp.integrate(f_dict[k1] * f_dict[k2], (s, 0, 1)))
            results[f'Cov_{k1}_{k2}'] = cov
    return results



In [4]:
def analyze_pauli_rotation(pauli_target):
    # Identify label dynamically
    inv_pauli_labels = {id(mat): label for label, mat in pauli_labels.items()}
    pauli_target_label = inv_pauli_labels.get(id(pauli_target), "Unknown")

    # Compute L_k and its Pauli decomposition
    Lk, f_components = compute_pauli_components(pauli_target)
    variances = compute_var_and_cov(f_components)

    # Print results
    print(f"\n=== Analysis for Pauli Target: {pauli_target_label} ===")
    print(f"\n--- Matrix L_k = U† {pauli_target_label} U ---")
    print(Lk)  # cleaner symbolic form


    print(f"\n--- Pauli Basis Components f_k^({pauli_target_label})(s) ---")
    for label, expr in f_components.items():
        print(f"f_{pauli_target_label}^{label}(s) =", expr)

    print(f"\n--- Variances and Covariances for L_k = {pauli_target_label} ---")
    for name, val in variances.items():
        print(f"{name} =", val)
        
    return Lk, f_components, variances


In [5]:
## Z - Target Depolarizing Component ##
Lk_Z, f_components_Z, variances_Z = analyze_pauli_rotation(Z)  # func call

# Extract relevant entries for cov_x matrix
VarZ_X = variances_Z['Var_X']
VarZ_Y = variances_Z['Var_Y']
VarZ_Z = variances_Z['Var_Z']
CovZ_XY = variances_Z.get('Cov_X_Y', 0)
CovZ_XZ = variances_Z.get('Cov_X_Z', 0)
CovZ_YZ = variances_Z.get('Cov_Y_Z', 0)


cov_Z_symbolic = sp.Matrix([[VarZ_X, CovZ_XY, CovZ_XZ],
                            [CovZ_XY, VarZ_Y, CovZ_YZ],
                            [CovZ_XZ, CovZ_YZ, VarZ_Z]])

cov_Z_symbolic = sp.simplify(cov_Z_symbolic)

cov_Z_symbolic


=== Analysis for Pauli Target: Z ===

--- Matrix L_k = U† Z U ---
Matrix([[cos(s*theta), 2*I*(I*sin(phi) - cos(phi))*sin(s*theta/2)*cos(s*theta/2)], [2*(-sin(phi) + I*cos(phi))*sin(s*theta/2)*cos(s*theta/2), -cos(s*theta)]])

--- Pauli Basis Components f_k^(Z)(s) ---
f_Z^I(s) = 0
f_Z^X(s) = -sin(phi)*sin(s*theta)
f_Z^Y(s) = sin(s*theta)*cos(phi)
f_Z^Z(s) = cos(s*theta)

--- Variances and Covariances for L_k = Z ---
Var_I = 0
Var_X = Piecewise(((theta - sin(2*theta)/2)*sin(phi)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True))
Var_Y = Piecewise(((theta - sin(2*theta)/2)*cos(phi)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True))
Var_Z = Piecewise(((theta + sin(2*theta)/2)/(2*theta), (theta > 0) | (theta < 0)), (1, True))
Cov_I_X = 0
Cov_I_Y = 0
Cov_I_Z = 0
Cov_X_Y = Piecewise(((-theta + sin(2*theta)/2)*sin(phi)*cos(phi)/(2*theta), (theta > 0) | (theta < 0)), (0, True))
Cov_X_Z = Piecewise((-sin(phi)*sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True))
Cov_Y_Z = Piecewis

Matrix([
[       Piecewise(((2*theta - sin(2*theta))*sin(phi)**2/(4*theta), (theta > 0) | (theta < 0)), (0, True)), Piecewise(((-2*theta + sin(2*theta))*sin(phi)*cos(phi)/(4*theta), (theta > 0) | (theta < 0)), (0, True)),  Piecewise((-sin(phi)*sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True))],
[Piecewise(((-2*theta + sin(2*theta))*sin(phi)*cos(phi)/(4*theta), (theta > 0) | (theta < 0)), (0, True)),        Piecewise(((2*theta - sin(2*theta))*cos(phi)**2/(4*theta), (theta > 0) | (theta < 0)), (0, True)),   Piecewise((sin(theta)**2*cos(phi)/(2*theta), (theta > 0) | (theta < 0)), (0, True))],
[                    Piecewise((-sin(phi)*sin(theta)**2/(2*theta), (theta > 0) | (theta < 0)), (0, True)),                      Piecewise((sin(theta)**2*cos(phi)/(2*theta), (theta > 0) | (theta < 0)), (0, True)), Piecewise(((2*theta + sin(2*theta))/(4*theta), (theta > 0) | (theta < 0)), (1, True))]])

In [6]:
## X - Target Depolarizing Component ##
# Compute and extract symbolic variances/covariances for Pauli X
Lk_X, f_components_X, variances_X = analyze_pauli_rotation(X)  # func call

# Extract relevant entries for cov_x matrix
VarX_X = variances_X['Var_X']
VarX_Y = variances_X['Var_Y']
VarX_Z = variances_X['Var_Z']
CovX_XY = variances_X.get('Cov_X_Y', 0)
CovX_XZ = variances_X.get('Cov_X_Z', 0)
CovX_YZ = variances_X.get('Cov_Y_Z', 0)

# Construct symbolic covariance matrix
cov_x_symbolic = sp.Matrix([
    [VarX_X,    CovX_XY,   CovX_XZ],
    [CovX_XY,   VarX_Y,    CovX_YZ],
    [CovX_XZ,   CovX_YZ,   VarX_Z]
])

cov_x_symbolic = sp.simplify(cov_x_symbolic)

cov_x_symbolic


=== Analysis for Pauli Target: X ===

--- Matrix L_k = U† X U ---
Matrix([[cos(phi - s*theta)/2 - cos(phi + s*theta)/2, cos(s*theta/2)**2 + exp(-2*I*phi)*sin(s*theta/2)**2], [exp(2*I*phi)*sin(s*theta/2)**2 + cos(s*theta/2)**2, -cos(phi - s*theta)/2 + cos(phi + s*theta)/2]])

--- Pauli Basis Components f_k^(X)(s) ---
f_X^I(s) = 0
f_X^X(s) = exp(2*I*phi)*sin(s*theta/2)**2/2 + cos(s*theta/2)**2 + exp(-2*I*phi)*sin(s*theta/2)**2/2
f_X^Y(s) = sin(2*phi)*sin(s*theta/2)**2
f_X^Z(s) = sin(phi)*sin(s*theta)

--- Variances and Covariances for L_k = X ---
Var_I = 0
Var_X = Piecewise(((4*theta*(3*exp(8*I*phi) + 4*exp(6*I*phi) + 18*exp(4*I*phi) + 4*exp(2*I*phi) + 3)*exp(2*I*(2*phi + theta)) + I*(8*(-exp(8*I*phi) + 2*exp(4*I*phi) - 1)*exp(I*theta) + ((-exp(8*I*phi) + 4*exp(6*I*phi) - 6*exp(4*I*phi) + 4*exp(2*I*phi) - 1)*exp(I*theta) + 8*exp(8*I*phi) - 16*exp(4*I*phi) + 8)*exp(3*I*theta) + exp(8*I*phi) - 4*exp(6*I*phi) + 6*exp(4*I*phi) - 4*exp(2*I*phi) + 1)*exp(4*I*phi))*exp(-2*I*(4*phi + theta))/(1

Matrix([
[Piecewise(((4*theta*(3*exp(8*I*phi) + 4*exp(6*I*phi) + 18*exp(4*I*phi) + 4*exp(2*I*phi) + 3)*exp(2*I*(2*phi + theta)) + I*(8*(-exp(8*I*phi) + 2*exp(4*I*phi) - 1)*exp(I*theta) + ((-exp(8*I*phi) + 4*exp(6*I*phi) - 6*exp(4*I*phi) + 4*exp(2*I*phi) - 1)*exp(I*theta) + 8*exp(8*I*phi) - 16*exp(4*I*phi) + 8)*exp(3*I*theta) + exp(8*I*phi) - 4*exp(6*I*phi) + 6*exp(4*I*phi) - 4*exp(2*I*phi) + 1)*exp(4*I*phi))*exp(-2*I*(4*phi + theta))/(128*theta), Ne(theta**4*exp(16*I*phi), 0)), (1, True)), Piecewise(((-4*I*theta*(3*exp(8*I*phi) + 2*exp(6*I*phi) - 2*exp(2*I*phi) - 3)*exp(2*I*(2*phi + theta)) + (8*(1 - exp(8*I*phi))*exp(I*theta) + ((-exp(8*I*phi) + 2*exp(6*I*phi) - 2*exp(2*I*phi) + 1)*exp(I*theta) + 8*exp(8*I*phi) - 8)*exp(3*I*theta) + exp(8*I*phi) - 2*exp(6*I*phi) + 2*exp(2*I*phi) - 1)*exp(4*I*phi))*exp(-2*I*(4*phi + theta))/(128*theta), Ne(theta**4*exp(16*I*phi), 0)), (0, True)), Piecewise((I*(-2*(3*exp(6*I*phi) + 7*exp(4*I*phi) - 7*exp(2*I*phi) - 3)*exp(3*I*(4*phi + theta)) - (4*(-exp

Check expression

In [7]:
# Substitute φ = pi/2 into each expression
phi_val = sp.pi / 2
subs_dict = {phi: phi_val}

VarX_X_num = VarX_X.subs(subs_dict).simplify()
VarX_Y_num = VarX_Y.subs(subs_dict).simplify()
VarX_Z_num = VarX_Z.subs(subs_dict).simplify()
CovX_XY_num = CovX_XY.subs(subs_dict).simplify()
CovX_XZ_num = CovX_XZ.subs(subs_dict).simplify()
CovX_YZ_num = CovX_YZ.subs(subs_dict).simplify()


# print all results
print(f"\n--- Numerical Values for X Target phi = 0 ---")
print(f"VarX_X = {VarX_X_num}")
print(f"VarX_Y = {VarX_Y_num}")
print(f"VarX_Z = {VarX_Z_num}")
print(f"CovX_XY = {CovX_XY_num}")
print(f"CovX_XZ = {CovX_XZ_num}")
print(f"CovX_YZ = {CovX_YZ_num}")

# === Numerical Simulation ===


--- Numerical Values for X Target phi = 0 ---
VarX_X = Piecewise((1/2 - I*exp(2*I*theta)/(8*theta) + I*exp(-2*I*theta)/(8*theta), (theta > 0) | (theta < 0)), (1, True))
VarX_Y = 0
VarX_Z = Piecewise(((2*theta - sin(2*theta))/(4*theta), (theta > 0) | (theta < 0)), (0, True))
CovX_XY = 0
CovX_XZ = Piecewise(((-exp(4*I*theta) + 2*exp(2*I*theta) - 1)*exp(-2*I*theta)/(8*theta), (theta > 0) | (theta < 0)), (0, True))
CovX_YZ = 0


In [8]:
## Y - Target Depolarizing Component ##
Lk_Y, f_components_Y, variances_Y = analyze_pauli_rotation(Y)

VarY_X = variances_Y['Var_X']
VarY_Y = variances_Y['Var_Y']
VarY_Z = variances_Y['Var_Z']
CovY_XY = variances_Y.get('Cov_X_Y', 0)
CovY_XZ = variances_Y.get('Cov_X_Z', 0)
CovY_YZ = variances_Y.get('Cov_Y_Z', 0)

cov_Y_symbolic = sp.Matrix([
    [VarY_X, CovY_XY, CovY_XZ],
    [CovY_XY, VarY_Y, CovY_YZ],
    [CovY_XZ, CovY_YZ, VarY_Z]
])

cov_Y_symbolic = sp.simplify(cov_Y_symbolic)

cov_Y_symbolic


=== Analysis for Pauli Target: Y ===

--- Matrix L_k = U† Y U ---
Matrix([[sin(phi - s*theta)/2 - sin(phi + s*theta)/2, I*(-cos(s*theta/2)**2 + exp(-2*I*phi)*sin(s*theta/2)**2)], [I*(-exp(2*I*phi)*sin(s*theta/2)**2 + cos(s*theta/2)**2), -sin(phi - s*theta)/2 + sin(phi + s*theta)/2]])

--- Pauli Basis Components f_k^(Y)(s) ---
f_Y^I(s) = 0
f_Y^X(s) = sin(2*phi)*sin(s*theta/2)**2
f_Y^Y(s) = -exp(2*I*phi)*sin(s*theta/2)**2/2 + cos(s*theta/2)**2 - exp(-2*I*phi)*sin(s*theta/2)**2/2
f_Y^Z(s) = -sin(s*theta)*cos(phi)

--- Variances and Covariances for L_k = Y ---
Var_I = 0
Var_X = Piecewise(((3*theta - 4*sin(theta) + sin(2*theta)/2)*sin(2*phi)**2/(8*theta), (theta > 0) | (theta < 0)), (0, True))
Var_Y = Piecewise(((4*theta*(3*exp(8*I*phi) - 4*exp(6*I*phi) + 18*exp(4*I*phi) - 4*exp(2*I*phi) + 3)*exp(2*I*(2*phi + theta)) + I*(8*(-exp(8*I*phi) + 2*exp(4*I*phi) - 1)*exp(I*theta) + (-(exp(8*I*phi) + 4*exp(6*I*phi) + 6*exp(4*I*phi) + 4*exp(2*I*phi) + 1)*exp(I*theta) + 8*exp(8*I*phi) - 16*exp(4*I*p

Matrix([
[                                                                                                                                                                                                                                                                                          Piecewise(((6*theta - 8*sin(theta) + sin(2*theta))*sin(2*phi)**2/(16*theta), (theta > 0) | (theta < 0)), (0, True)),                                                                                       Piecewise(((4*I*theta*(3*exp(8*I*phi) - 2*exp(6*I*phi) + 2*exp(2*I*phi) - 3)*exp(2*I*(2*phi + theta)) + (-8*(1 - exp(8*I*phi))*exp(I*theta) + (-(-exp(8*I*phi) - 2*exp(6*I*phi) + 2*exp(2*I*phi) + 1)*exp(I*theta) - 8*exp(8*I*phi) + 8)*exp(3*I*theta) - exp(8*I*phi) - 2*exp(6*I*phi) + 2*exp(2*I*phi) + 1)*exp(4*I*phi))*exp(-2*I*(4*phi + theta))/(128*theta), Ne(theta**4*exp(16*I*phi), 0)), (0, True)),                                                                                                           

Check that this matches Noisy Gate Library Code

In [9]:
# Substitute φ = 0 into each expression
phi_val = 0
subs_dict = {phi: phi_val}

VarY_X_num = VarY_X.subs(subs_dict).simplify()
VarY_Y_num = VarY_Y.subs(subs_dict).simplify()
VarY_Z_num = VarY_Z.subs(subs_dict).simplify()
CovY_XY_num = CovY_XY.subs(subs_dict).simplify()
CovY_XZ_num = CovY_XZ.subs(subs_dict).simplify()
CovY_YZ_num = CovY_YZ.subs(subs_dict).simplify()

# print all results
print(f"\n--- Numerical Values for Y Target phi = 0 ---")
print(f"VarY_X = {VarY_X_num}")
print(f"VarY_Y = {VarY_Y_num}")
print(f"VarY_Z = {VarY_Z_num}")
print(f"CovY_XY = {CovY_XY_num}")
print(f"CovY_XZ = {CovY_XZ_num}")
print(f"CovY_YZ = {CovY_YZ_num}")
# === Numerical Simulation ===


--- Numerical Values for Y Target phi = 0 ---
VarY_X = 0
VarY_Y = Piecewise((1/2 - I*exp(2*I*theta)/(8*theta) + I*exp(-2*I*theta)/(8*theta), (theta > 0) | (theta < 0)), (1, True))
VarY_Z = Piecewise(((2*theta - sin(2*theta))/(4*theta), (theta > 0) | (theta < 0)), (0, True))
CovY_XY = 0
CovY_XZ = 0
CovY_YZ = Piecewise(((exp(4*I*theta) - 2*exp(2*I*theta) + 1)*exp(-2*I*theta)/(8*theta), (theta > 0) | (theta < 0)), (0, True))


## Compute Depolarizing Noise
Now neeed to rewrite the noise matricies 

In [10]:
# compute depolarizing channel for X component
def sample_pauli_noise_matrix(phi_val, theta_val, ed, cov_σ_sym, phi_sym, theta_sym):
    """
    Sample a Hermitian Pauli noise matrix aligned with Paulis, 
    using φ- and θ-dependent covariance matrix.

    Parameters:
        phi_val     : float — φ angle (rotation axis)
        theta_val   : float — θ angle (rotation amplitude)
        ed          : float — depolarizing strength amplitude
        cov_σ_sym   : sympy.Matrix — symbolic covariance matrix with entries depending on phi, theta
        phi_sym     : sympy.Symbol — symbolic phi variable used in cov_x_sym
        theta_sym   : sympy.Symbol — symbolic theta variable used in cov_x_sym

    Returns:
        Idσ : np.ndarray — 2x2 Hermitian noise matrix
    """
    # Evaluate symbolic covariance matrix numerically
    cov_σ_numeric = np.array(cov_σ_sym.evalf(subs={phi_sym: phi_val, theta_sym: theta_val})).astype(np.complex128)

    # Sample from multivariate normal with zero mean
    mean_σ = np.zeros(3)
    Xc, Yc, Zc = np.random.multivariate_normal(mean_σ, cov_σ_numeric, 1)[0]

    # Build Hermitian matrix
    Idσ = ed * np.array([
        [ Zc,       Xc - 1j * Yc ],
        [ Xc + 1j * Yc,   -Zc     ]
    ], dtype=np.complex128)

    return Idσ

# Sample from φ-dependent version
def sample_pauli_noise_matrix_numeric(ed, cov_σ_numeric):
    mean = np.zeros(3)
    Xc, Yc, Zc = np.random.multivariate_normal(mean, cov_σ_numeric, 1)[0]
    Idσ = ed * np.array([
        [ Zc,       Xc - 1j * Yc ],
        [ Xc + 1j * Yc,   -Zc     ]
    ], dtype=np.complex128)
    return Idσ


Check With Library Version

In [11]:
def depolarization_noise_x(theta,phi,ed):
    # Variances and covariances for depolarization Itô processes (X axis)
    Vdx_1 = (2*theta - np.sin(2*theta)) / (4*theta)
    Vdx_2 = (6*theta - 8*np.sin(theta) + np.sin(2*theta)) / (16*theta)
    Covdx_12 = (np.sin(theta/2))**4 / theta
    Covdx_1Wdx = (1 - np.cos(theta)) / theta
    Covdx_2Wdx = (theta - np.sin(theta)) / (2 * theta)

    meand_x = array([0, 0, 0])
    covd_x = array([
        [Vdx_1, Covdx_12, Covdx_1Wdx],
        [Covdx_12, Vdx_2, Covdx_2Wdx],
        [Covdx_1Wdx, Covdx_2Wdx, 1]
    ])

    sample_dx = multivariate_normal(meand_x, covd_x, 1)
    Idx1, Idx2, Wdx = sample_dx[0]

    Idx = ed * array([
        [np.sin(phi) * Idx1, Wdx + (exp(-2j*phi) - 1) * Idx2],
        [Wdx + (exp(2j*phi) - 1) * Idx2, -np.sin(phi) * Idx1]
    ])
    return Vdx_1, Vdx_2, Covdx_12, Covdx_1Wdx, Covdx_2Wdx, Idx


In [26]:
# Define symbols
# Set φ and θ values
phi_val = 0
theta_val = sp.pi/2
ed = 0.1
subs_dict = {phi: phi_val, theta: theta_val}
# Substitute φ and θ into the covariance matrix

Vdx_1_lib, Vdx_2_lib, Covdx_12_lib, Covdx_1Wdx_lib, Covdx_2Wdx_lib, Idx_lib = depolarization_noise_x(float(theta_val), float(phi_val), ed)


# Print results
print(f"\n--- Library Numerical Values for Depolarization Noise ---")
print(f"Vdx_1 = {Vdx_1_lib}")
print(f"Vdx_2 = {Vdx_2_lib}")
print(f"Covdx_12 = {Covdx_12_lib}")
print(f"Covdx_1Wdx = {Covdx_1Wdx_lib}")
print(f"Covdx_2Wdx = {Covdx_2Wdx_lib}")

print(f"Idx_lib = {Idx_lib}")

# === Numerical Solutions ===


VarX_X_numer = VarX_X.subs(subs_dict).simplify()
VarX_Y_numer = VarX_Y.subs(subs_dict).simplify()
VarX_Z_numer = VarX_Z.subs(subs_dict).simplify()
CovX_XY_numer = CovX_XY.subs(subs_dict).simplify()
CovX_XZ_numer = CovX_XZ.subs(subs_dict).simplify()
CovX_YZ_numer = CovX_YZ.subs(subs_dict).simplify()

cov_x_numer = sp.Matrix([
    [VarX_X_numer, CovX_XY_numer, CovX_XZ_numer],
    [CovX_XY_numer, VarX_Y_numer, CovX_YZ_numer],
    [CovX_XZ_numer, CovX_YZ_numer, VarX_Z_numer]
])


# print all results
print(f"\n--- Numerical Values for X Target phi = {phi_val} ---")
print(f"VarX_X = {VarX_X_numer}")
print(f"VarX_Y = {VarX_Y_numer}")
print(f"VarX_Z = {VarX_Z_numer}")
print(f"CovX_XY = {CovX_XY_numer}")
print(f"CovX_XZ = {CovX_XZ_numer}")
print(f"CovX_YZ = {CovX_YZ_numer}")

Idx_numer = sample_pauli_noise_matrix_numeric(ed, cov_x_numer)
print(f"Idx_numer = {Idx_numer}")



--- Library Numerical Values for Depolarization Noise ---
Vdx_1 = 0.5
Vdx_2 = 0.05669011381620932
Covdx_12 = 0.15915494309189537
Covdx_1Wdx = 0.6366197723675813
Covdx_2Wdx = 0.1816901138162093
Idx_lib = [[ 0.        +0.j  0.24664987+0.j]
 [ 0.24664987+0.j -0.        +0.j]]

--- Numerical Values for X Target phi = 0 ---
VarX_X = 1
VarX_Y = 0
VarX_Z = 0
CovX_XY = 0
CovX_XZ = 0
CovX_YZ = 0
Idx_numer = [[ 0.        +0.j -0.00219295+0.j]
 [-0.00219295+0.j -0.        +0.j]]


In [27]:
num_samples = 10000
samples_phi = []
samples_lib = []

for _ in range(num_samples):
    # φ-dependent sampling
    cov_phi_numeric = np.array(cov_x_symbolic.evalf(subs={phi: phi_val, theta: theta_val})).astype(np.float64)
    Xc, Yc, Zc = np.random.multivariate_normal([0, 0, 0], cov_phi_numeric)
    samples_phi.append([Xc, Yc, Zc])

    # Library sampling
    Vdx_1, Vdx_2, Covdx_12, Covdx_1Wdx, Covdx_2Wdx, _ = depolarization_noise_x(float(theta_val), float(phi_val), ed)
    cov_lib = np.array([
        [Vdx_1, Covdx_12, Covdx_1Wdx],
        [Covdx_12, Vdx_2, Covdx_2Wdx],
        [Covdx_1Wdx, Covdx_2Wdx, 1.0]
    ])
    Idx1, Idx2, Wdx = np.random.multivariate_normal([0, 0, 0], cov_lib)
    samples_lib.append([
        np.sin(float(phi_val)) * Idx1,
        (np.exp(2j * float(phi_val)) - 1).real * Idx2,  # approximate
        Wdx
    ])


In [28]:
samples_phi = np.array(samples_phi)
samples_lib = np.array(samples_lib)

mean_phi = np.mean(samples_phi, axis=0)
cov_phi = np.cov(samples_phi.T)

mean_lib = np.mean(samples_lib, axis=0)
cov_lib = np.cov(samples_lib.T)

print("Mean diff:", np.abs(mean_phi - mean_lib))
print("Covariance diff:\n", np.abs(cov_phi - cov_lib))


Mean diff: [0.01013072 0.         0.0043387 ]
Covariance diff:
 [[1.02404812 0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         1.01893266]]


WHAT THE FCCKKKK

In [19]:
def depolarization_noise_z(theta,phi,ed):
    Vdz_1 = (2*theta + np.sin(2*theta)) / (4*theta)
    Vdz_2 = (2*theta - np.sin(2*theta)) / (4*theta)
    Covdz_12 = (np.sin(theta))**2 / (2*theta)

    meand_z = array([0, 0])
    covd_z = array([
        [Vdz_1, Covdz_12],
        [Covdz_12, Vdz_2]
    ])

    sample_dz = multivariate_normal(meand_z, covd_z, 1)
    Idz1, Idz2 = sample_dz[0]

    Idz = ed * array([
        [Idz1, -1j * exp(-1j*phi) * Idz2],
        [1j * exp(1j*phi) * Idz2, -Idz1]
    ])
    return Vdz_1, Vdz_2, Covdz_12, Idz


In [20]:
# Define symbols
# Set φ and θ values
phi_val = 0
theta_val = sp.pi
ed = 0.1
subs_dict = {phi: phi_val, theta: theta_val}
# Substitute φ and θ into the covariance matrix

Vdz_1, Vdz_2, Covdz_12, Idz = depolarization_noise_z(float(theta_val), float(phi_val), ed)


# Print results
print(f"\n--- Library Numerical Values for Depolarization Noise ---")
print(f"Vdz_1 = {Vdz_1}")
print(f"Vdz_2 = {Vdz_2}")
print(f"Covdz_12 = {Covdz_12}")
print(f"Idz = {Idz}")

# === Numerical Solutions ===
VarZ_X_numer = VarZ_X.subs(subs_dict).simplify()
VarZ_Y_numer = VarZ_Y.subs(subs_dict).simplify()
VarZ_Z_numer = VarZ_Z.subs(subs_dict).simplify()
CovZ_XY_numer = CovZ_XY.subs(subs_dict).simplify()
CovZ_XZ_numer = CovZ_XZ.subs(subs_dict).simplify()
CovZ_YZ_numer = CovZ_YZ.subs(subs_dict).simplify()

cov_Z_numer = sp.Matrix([
    [VarZ_X_numer, CovZ_XY_numer, CovZ_XZ_numer],
    [CovZ_XY_numer, VarZ_Y_numer, CovZ_YZ_numer],
    [CovZ_XZ_numer, CovZ_YZ_numer, VarZ_Z_numer]
])


# print all results
print(f"\n--- Numerical Values for Z Target phi = {phi_val} ---")
print(f"VarZ_X = {VarZ_X_numer}")
print(f"VarZ_Y = {VarZ_Y_numer}")
print(f"VarZ_Z = {VarZ_Z_numer}")
print(f"CovZ_XY = {CovZ_XY_numer}")
print(f"CovZ_XZ = {CovZ_XZ_numer}")
print(f"CovZ_YZ = {CovZ_YZ_numer}") 



Idz_numer = sample_pauli_noise_matrix_numeric(ed=0.1, cov_σ_numeric=cov_Z_numer)

print(f"\n\nIdx_numeric = {Idz_numer}")


--- Library Numerical Values for Depolarization Noise ---
Vdz_1 = 0.5
Vdz_2 = 0.5
Covdz_12 = 2.3869418286106128e-33
Idz = [[ 0.08760467+0.00000000e+00j  0.06332237+3.87737692e-18j]
 [ 0.06332237-3.87737692e-18j -0.08760467+0.00000000e+00j]]

--- Numerical Values for Z Target phi = pi/2 ---
VarZ_X = 1/2
VarZ_Y = 0
VarZ_Z = 1/2
CovZ_XY = 0
CovZ_XZ = 0
CovZ_YZ = 0


Idx_numeric = [[-0.17041195+0.j -0.00062256+0.j]
 [-0.00062256+0.j  0.17041195+0.j]]
